In [1]:
import os
import base64

from dotenv import load_dotenv
from openai import AzureOpenAI
from mimetypes import guess_type

import pandas as pd

To do:

- Improve prompt.
- Run on all images.

In [2]:
os.chdir("../")
CWD = os.getcwd()

data_dir = os.path.join(CWD, 'data')

In [18]:
image_path = os.path.join(data_dir, 'proc', 'building_plans','split_pdf', '9993_0.jpg')

In [19]:
def encode_image_to_base64(image_path):
    # Open the image file in binary mode
    with open(image_path, "rb") as image_file:
        # Read the binary data
        image_data = image_file.read()
        # Encode the binary data to base64
        encoded_image = base64.b64encode(image_data)
        # Convert bytes to string
        encoded_image_str = encoded_image.decode('utf-8')
        return encoded_image_str


In [20]:
base64_images = encode_image_to_base64(image_path)

In [7]:
# Option 1: Use httpimport to load 'azure_authentication' package remotely from GitHub without installing it
import httpimport
with httpimport.remote_repo('https://raw.githubusercontent.com/soda-lmu/azure-auth-helper-python/main/src'
                            '/azure_authentication/'):
    from customized_azure_login import CredentialFactory

In [8]:
DEPLOYMENT_NAME = "gpt-4-turbo-vision-preview"

print("Authenticate User & Login to Azure Cognitive Services")
# Recommendation: Configure your own authentication workflow with environment variables, see the description at
# https://github.com/soda-lmu/azure-auth-helper-python/blob/main/AuthenticationWorkflowSetup.md
credential = CredentialFactory().select_credential()
token_provider = credential.get_login_token_to_azure_cognitive_services()

print("Instantiate Azure OpenAI Client")

Authenticate User & Login to Azure Cognitive Services
Instantiate Azure OpenAI Client


In [27]:
#token_provider()

In [10]:
client = AzureOpenAI(
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
    api_key=token_provider(),  # alternative: insert os.getenv("AZURE_OPENAI_API_KEY")
    # azure_ad_token=token_provider(),          # same outcome
    # azure_ad_token_provider=token_provider,   # same outcome again
    api_version="2024-02-01",  # or use a preview version (e.g., "2024-03-01-preview") for the latest features.
    # api_version (How-To): https://stackoverflow.com/questions/76475419/how-can-i-select-the-proper-openai-api-version
    timeout=600.0,  # throw APITimeoutError after 10 minutes without a response (default behavior)
)

In [24]:
response = client.chat.completions.create(
  model=DEPLOYMENT_NAME,
  messages=[
      {"role": "system",
       "content": "You are a helpful urban planning analyst."
       },
    {"role": "user",
      "content": [
        {"type": "text", 
         "text": "Extract the different elements specified in the Festsetzungen of the plan."},
        {
          "type": "image_url",
          "image_url": {
            "url": f"data:image/jpeg;base64,{base64_images}",
          },
        },
      ],
    }
  ],
  max_tokens=1000,
)

In [25]:
print(response.choices[0].message.content)

The plan image provided is a zoning plan (Bebauungsplan) from the Gemeinde Blankenbach, Landkreis Aschaffenburg which outlines specific planning regulations for development within the designated area. This kind of plan typically includes a map as well as textual regulations (textliche Festsetzungen) that determine how land can be used and what kind of structures can be built.

From the image, the following elements can be extracted:

**Planzeichenerklärung (Key to symbols)**:
- **Flächen für den Gemeinbedarf** (Areas for community use), including Zweckbestimmung (specific purpose) such as Festplatz, Bolzplatz, Spielplatz, Bauhof.
- **Spielplatz** (Playground).
- **Bauhof** (Yard).
- Symbols for **Grenze des räumlichen Geltungsbereiches** (Boundary of the spatial scope), **Abgrenzung unterschiedlicher Nutzung** (Differentiation of different uses), etc.

**Teil A: Planungsrechtliche Festsetzungen (Part A: Legal planning regulations)**:
- **Flächen für den Gemeinbedarf** (Area for public 